<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Collagen-Pradeep/Bioco_Collagen_Klassiske_Superveiledet_Regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lasting av nødvendige bibliotek og pakker

In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 5.1 MB/s eta 0:00:00


Importering av nødvendig bibliotek og pakker

In [2]:
import pandas as pd
import numpy as np
import copy
import math

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error, r2_score

import optuna
from optuna.visualization import plot_optimization_history

Importering av relevant data

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/MyDrive/MasterV24/GitHub/
%ls
#from oppdeling_trening_testsett import oppdeling_enzymtype

/content/drive/MyDrive/MasterV24/GitHub
'Bioco_BrixAdjusted: Semiveiledet Regresjonsmetoder.ipynb'
'Bioco_BrixAdjusted: Superveiledet Regresjonsmetoder.ipynb'
'Bioco_Collagen: Klassiske Superveiledet Regresjonsmetoder.ipynb'
'Bioco_Collagen: Semiveiledet Regresjonsmetoder.ipynb'
'Bioco: Databehandling av ekstremverdier.ipynb'
'Bioco: Databehandling av manglende verdier.ipynb'
'Bioco: Dataprosessering, Dataforståelse og Dataundersøkelse.ipynb'
'Bioco_Mw: Semiveiledet Regresjonsmetoder.ipynb'
'Bioco_Mw: Superveiledet Regresjonsmetoder.ipynb'
'Bioco_Smallmolecules: Semiveiledet Regresjonsmetoder.ipynb'
'Bioco_Smallmolecules: Superveiledet Regresjonsmetoder.ipynb'
'Bioco: Variabelundersøkelse.ipynb'
 Oppdeling_trening_testsett.ipynb
 oppdeling_trening_testsett.py


In [5]:
# Velger første kolonne med dato og tid som index
collagen_data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data.csv',
                            header=0,
                            sep=',',
                            index_col=0)

# Formatterer index til riktig format og datatype
collagen_data.index = pd.to_datetime(collagen_data.index,
                                     format='%Y-%m-%d %H:%M:%S')

Beskrivelse av datasett

In [6]:
collagen_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29136 entries, 2022-10-31 17:37:00 to 2023-06-14 01:06:00
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  29136 non-null  int64  
 1   EnzymeType_A2  29136 non-null  int64  
 2   EnzymeType_B   29136 non-null  int64  
 3   EnzymeType_C   29136 non-null  int64  
 4   EnzymeType_D   29136 non-null  int64  
 5   EnzymeType_E   29136 non-null  int64  
 6   RawMatFlow     29136 non-null  float64
 7   NIRfat         29136 non-null  float64
 8   NIRash         29136 non-null  float64
 9   NIRwater       29136 non-null  float64
 10  TT07           29136 non-null  float64
 11  TT08           29136 non-null  float64
 12  PT03           29136 non-null  float64
 13  TT20           29136 non-null  float64
 14  TT09           29136 non-null  float64
 15  TT12           29136 non-null  float64
 16  Collagen       89 non-null     float64
dtypes: float64(11),

Tilfeldighetsfrø

In [7]:
#Tilfeldighetsfrø
random_seed = 123

# NIR målinger inkludert

## Oppdeling av datasett for trening og testing

Markert del av datasett

In [9]:
# Deler opp markert del av datasettet
collagen_markert = collagen_data.dropna(subset='Collagen')

### Alternativ 1: Fordelt på Enzymtyper

In [ ]:
# Legger til kategorisk variabel for enzymtype til fordeling av datsettet
collagen_markert['EnzymType'] = collagen_markert.filter(like='EnzymeType_').idxmax(axis=1).str.split('_').str[1].astype('category')

# Splitter datasett i collagendataasett og kategorisk enzymtype
collagen_enzymetypes = collagen_markert['EnzymType']
collagen_markert.drop(columns=['EnzymType'], inplace=True)

# Splitter trening og testdata etter enzymtype
collagen_trening, collagen_test, _, _ = train_test_split(collagen_markert,
                                                         collagen_enzymetypes,
                                                         test_size = 0.20,
                                                         stratify = collagen_enzymetypes,
                                                         random_state = random_seed)

# Deler opp datasettene til prediktorer og respons, for trening og testsett
X_trening = collagen_trening.iloc[:, :-1]
X_test = collagen_test.iloc[:, :-1]
y_trening = collagen_trening.iloc[:, -1]
y_test = collagen_test.iloc[:, -1]

<ipython-input-10-33b276e70c93>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collagen_markert['EnzymType'] = collagen_markert.filter(like='EnzymeType_').idxmax(axis=1).str.split('_').str[1].astype('category')
<ipython-input-10-33b276e70c93>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collagen_markert.drop(columns=['EnzymType'], inplace=True)


**RandomForestRegressor**

Hyperparamter optimalisering med Optuna

In [ ]:
def objective(trial):
  parametere = {
      'n_estimators': trial.suggest_int('n_estimators', 100, 200),
      'max_depth': trial.suggest_int('max_depth', 1, 10)
  }
  rf_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**parametere, random_state=random_seed))
  ])

  rf_pipeline.fit(X_trening, y_trening)
  y_test_prediksjon = rf_pipeline.predict(X_test)
  mse_test = mean_squared_error(y_test, y_test_prediksjon)
  return mse_test

if __name__ == "__main__":
  study = optuna.create_study(direction='minimize')
  study.optimize(objective, n_trials=100)

plot_optimization_history(study)

[I 2024-02-09 10:26:32,020] A new study created in memory with name: no-name-f2af1e57-f706-4f52-9819-26a32960ee35
[I 2024-02-09 10:26:32,179] Trial 0 finished with value: 19.6704640380865 and parameters: {'n_estimators': 107, 'max_depth': 2}. Best is trial 0 with value: 19.6704640380865.
[I 2024-02-09 10:26:32,442] Trial 1 finished with value: 16.400831127512316 and parameters: {'n_estimators': 165, 'max_depth': 8}. Best is trial 1 with value: 16.400831127512316.
[I 2024-02-09 10:26:32,722] Trial 2 finished with value: 16.445357463104088 and parameters: {'n_estimators': 167, 'max_depth': 9}. Best is trial 1 with value: 16.400831127512316.
[I 2024-02-09 10:26:32,979] Trial 3 finished with value: 16.612291952282423 and parameters: {'n_estimators': 167, 'max_depth': 7}. Best is trial 1 with value: 16.400831127512316.
[I 2024-02-09 10:26:33,195] Trial 4 finished with value: 16.86672884499781 and parameters: {'n_estimators': 133, 'max_depth': 4}. Best is trial 1 with value: 16.4008311275123

Evauleringer av beste modell

In [ ]:
def detailed_objective(trial):
  parametere = {
      'n_estimators': trial.suggest_int('n_estimators', 100, 200),
      'max_depth': trial.suggest_int('max_depth', 1, 10)
  }
  rf_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**parametere, random_state=random_seed))
  ])

  rf_pipeline.fit(X_trening, y_trening)
  y_test_prediksjon = rf_pipeline.predict(X_test)
  y_trening_prediksjon = rf_pipeline.predict(X_trening)

  mse_test = mean_squared_error(y_test, y_test_prediksjon)
  r2_test = r2_score(y_test, y_test_prediksjon)

  mse_trening = mean_squared_error(y_trening, y_trening_prediksjon)
  r2_trening = r2_score(y_trening, y_trening_prediksjon)

  return mse_test, r2_test, mse_trening, r2_trening

mse_test_resultat = detailed_objective(study.best_trial)[0]
r2_test_resultat = detailed_objective(study.best_trial)[1]
mse_trening_resultat = detailed_objective(study.best_trial)[2]
r2_trening_resultat = detailed_objective(study.best_trial)[3]

print("Beste hyperparametere for RandomForestRegressor: {}".format(study.best_params))
print("Test: MSE: {}, og R^2: {}".format(mse_test_resultat, r2_test_resultat))
print("Trening: MSE: {}, og R^2: {}".format(mse_trening_resultat, r2_trening_resultat))

Beste hyperparametere for RandomForestRegressor: {'n_estimators': 120, 'max_depth': 8}
Test: MSE: 16.15980740280381, og R^2: 0.5349908701216666
Trening: MSE: 1.6385169427242472, og R^2: 0.935338037139651


### Alternativ 2: Fordelt på ukentlig og daglig variasjon i produksjon m/kontinuitet

In [10]:
# Lager nytt trening og testsett
collagen_trening2 = copy.deepcopy(collagen_markert)
collagen_test2 = pd.DataFrame(columns = list(collagen_trening2.columns))

# Definere andel av datasettet som testsett
andel = 0.2

# Går gjennom hver dag, hver uke og samler første andel av rader til testsett
for uke in collagen_markert.index.isocalendar().week.unique():
  for dag in collagen_markert.index.isocalendar().day.unique():
    data = collagen_markert[(collagen_markert.index.isocalendar().week == uke) &
     (collagen_markert.index.isocalendar().day == dag)]
    test_rader = (data.shape[0])*andel
    test_data_dag = data.head(round(test_rader))

    collagen_test2 = pd.concat([collagen_test2, test_data_dag])
    collagen_trening2.drop(test_data_dag.index, inplace=True)

In [11]:
# Deler opp datasettene til prediktorer og respons, for trening og testsett
X_trening2 = collagen_trening2.iloc[:, :-1]
X_test2 = collagen_test2.iloc[:, :-1]
y_trening2 = collagen_trening2.iloc[:, -1]
y_test2 = collagen_test2.iloc[:, -1]

**RandomForestRegressor**

Hyperparameteroptimalisering med Optuna

In [12]:
def objective(trial):
  parametere = {
      'n_estimators': trial.suggest_int('n_estimators', 100, 200),
      'max_depth': trial.suggest_int('max_depth', 1, 10)
  }
  rf_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**parametere, random_state=random_seed))
  ])

  rf_pipeline.fit(X_trening2, y_trening2)
  y_test_prediksjon = rf_pipeline.predict(X_test2)
  mse_test = mean_squared_error(y_test2, y_test_prediksjon)
  return mse_test

if __name__ == "__main__":
  study = optuna.create_study(direction='minimize')
  study.optimize(objective, n_trials=100)

plot_optimization_history(study)

[I 2024-02-11 12:18:12,104] A new study created in memory with name: no-name-3cde0265-b0d7-4560-83e8-18b25eb39363
[I 2024-02-11 12:18:12,276] Trial 0 finished with value: 8.892155727103837 and parameters: {'n_estimators': 102, 'max_depth': 4}. Best is trial 0 with value: 8.892155727103837.
[I 2024-02-11 12:18:12,450] Trial 1 finished with value: 8.501761378974273 and parameters: {'n_estimators': 111, 'max_depth': 10}. Best is trial 1 with value: 8.501761378974273.
[I 2024-02-11 12:18:12,612] Trial 2 finished with value: 13.767718751686687 and parameters: {'n_estimators': 120, 'max_depth': 1}. Best is trial 1 with value: 8.501761378974273.
[I 2024-02-11 12:18:12,811] Trial 3 finished with value: 10.902552564959757 and parameters: {'n_estimators': 139, 'max_depth': 2}. Best is trial 1 with value: 8.501761378974273.
[I 2024-02-11 12:18:13,050] Trial 4 finished with value: 13.341519637902394 and parameters: {'n_estimators': 177, 'max_depth': 1}. Best is trial 1 with value: 8.50176137897427

In [13]:
def detailed_objective(trial):
  parametere = {
      'n_estimators': trial.suggest_int('n_estimators', 100, 200),
      'max_depth': trial.suggest_int('max_depth', 1, 10)
  }
  rf_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**parametere, random_state=random_seed))
  ])

  rf_pipeline.fit(X_trening2, y_trening2)
  y_test_prediksjon = rf_pipeline.predict(X_test2)
  y_trening_prediksjon = rf_pipeline.predict(X_trening2)

  mse_test = mean_squared_error(y_test2, y_test_prediksjon)
  r2_test = r2_score(y_test2, y_test_prediksjon)

  mse_trening = mean_squared_error(y_trening2, y_trening_prediksjon)
  r2_trening = r2_score(y_trening2, y_trening_prediksjon)

  return mse_test, r2_test, mse_trening, r2_trening

mse_test_resultat = detailed_objective(study.best_trial)[0]
r2_test_resultat = detailed_objective(study.best_trial)[1]
mse_trening_resultat = detailed_objective(study.best_trial)[2]
r2_trening_resultat = detailed_objective(study.best_trial)[3]

print("Beste hyperparametere for RandomForestRegressor: {}".format(study.best_params))
print("Test: MSE: {}, og R^2: {}".format(mse_test_resultat, r2_test_resultat))
print("Trening: MSE: {}, og R^2: {}".format(mse_trening_resultat, r2_trening_resultat))

Beste hyperparametere for RandomForestRegressor: {'n_estimators': 197, 'max_depth': 7}
Test: MSE: 8.079743300024674, og R^2: 0.6577768822921066
Trening: MSE: 1.817864367572221, og R^2: 0.9351151886016815


# NIR målinger ekskludert